1

In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [ ]:
df = pd.read_csv("Dataset_A_loan.csv")
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
data = df.copy()
categorical_cols = ['person_gender', 'person_education', 'person_home_ownership',
                    'loan_intent', 'previous_loan_defaults_on_file']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
data.dropna(inplace=True)
X = data.drop('loan_status', axis=1)
y = data['loan_status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_preds = rf_model.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_preds)
print("Random Forest Accuracy:", rf_acc)

Random Forest Accuracy: 0.9305263157894736


In [ ]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_scaled, y_train)
xgb_preds = xgb_model.predict(X_test_scaled)
xgb_acc = accuracy_score(y_test, xgb_preds)
print("XGBoost Accuracy:", xgb_acc)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:33:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.9334502923976609


In [ ]:
best_model = rf_model if rf_acc >= xgb_acc else xgb_model
best_model_name = "Random Forest" if rf_acc >= xgb_acc else "XGBoost"
print("Best model:", best_model_name)

with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

Best model: XGBoost


In [ ]:
import xgboost as xgb
import pickle

# Misal model sudah dilatih
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Simpan model
with open("best_model.pkl", "wb") as file:
    pickle.dump(model, file)


In [ ]:
import pickle

# Ganti nama variabel jika model XGBoost-mu berbeda
best_xgb = xgb_model  # atau model_xgb, clf_xgb, dsb.

with open("best_model.pkl", "wb") as f:
    pickle.dump(best_xgb, f)

print("✔ best_model.pkl berhasil dibuat")


✔ best_model.pkl berhasil dibuat


In [ ]:
from google.colab import files
files.download('best_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import pickle

# Load dataset
df = pd.read_csv("Dataset_A_loan.csv")

# Preprocessing
df = df.dropna()

# Encode categorical variables (penyederhanaan)
df['person_home_ownership'] = df['person_home_ownership'].replace('OTHER', 'RENT')
df['loan_intent'] = df['loan_intent'].replace({
    'DEBTCONSOLIDATION': 'PERSONAL',
    'HOMEIMPROVEMENT': 'PERSONAL'
})

X = df.drop('loan_status', axis=1)
y = df['loan_status']

# One-hot encoding
X = pd.get_dummies(X)
expected_columns = X.columns.tolist()

# Split & scaling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train_scaled, y_train)

# Simpan model
with open("best_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Simpan scaler dan kolom
with open("scaler.pkl", "wb") as f:
    pickle.dump((scaler, expected_columns), f)

print("✅ File 'best_model.pkl' dan 'scaler.pkl' berhasil disimpan.")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:19:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ File 'best_model.pkl' dan 'scaler.pkl' berhasil disimpan.
